In [2]:
# 특정 카테고리에 집중해서 판매하는 셀러의 매출이 높을지, 여러 카테고리 판매하는 셀러의 매출이 높을지

import pandas as pd

customers = pd.read_csv('../customers.csv')
locations = pd.read_csv('../locations.csv')
order_items = pd.read_csv('../order_items.csv')
orders = pd.read_csv('../orders.csv')
payments = pd.read_csv('../payments.csv')
products = pd.read_csv('../products.csv')
sellers = pd.read_csv('../sellers.csv')

# Merge orders with order_items to calculate revenue per day
orders_items_merged = pd.merge(orders, order_items, on='Order_id')

# Merge order _items with products to include product category information
seller_product_data = pd.merge(orders_items_merged, products, on='Product_id')

# Calculate the revenue considering the quantity of each item sold
seller_product_data['Revenue'] = seller_product_data['Price'] * seller_product_data['Order_item_id']

# Calculate total revenue and category count per seller
seller_performance_analysis = seller_product_data.groupby('Seller_id').agg({
    'Product_category_name': pd.Series.nunique,  # Count of unique categories per seller
    'Revenue': 'sum'  # Sum of revenue considering quantity
}).reset_index()

# Rename columns for clarity
seller_performance_analysis.columns = ['Seller_id', 'Category_Count', 'Total_Revenue']

# Analyze the relationship between the number of categories a seller handles and their total revenue
correlation_analysis = seller_performance_analysis[['Category_Count', 'Total_Revenue']].corr()
seller_performance_analysis.head(), correlation_analysis
d

(     Seller_id  Category_Count  Total_Revenue
 0  SELLER_0000               1         229.50
 1  SELLER_0001               3       11643.07
 2  SELLER_0002               1         167.99
 3  SELLER_0003               1         179.70
 4  SELLER_0004               6        3644.25,
                 Category_Count  Total_Revenue
 Category_Count        1.000000       0.305067
 Total_Revenue         0.305067       1.000000)

셀러별로 판매하는 상품 카테고리 수와 그들의 총 순이익 데이터를 분석한 결과, 상품 카테고리 수와 총 순이익 사이에 약간의 양의 상관관계(상관계수 약 0.305)가 있음을 확인할 수 있습니다. 이는 여러 카테고리의 상품을 판매하는 셀러가 일반적으로 더 높은 순이익을 올릴 가능성이 있음을 시사합니다.

분석 결과:
카테고리 수와 순이익의 관계: 다양한 카테고리의 상품을 취급하는 셀러들이 한정된 카테고리의 상품만 취급하는 셀러보다 높은 순이익을 얻는 경향이 있습니다. 이는 다양한 상품을 제공함으로써 더 많은 고객의 요구를 충족시키고 매출을 다각화할 수 있기 때문일 수 있습니다.
이 결과를 바탕으로, 판매 전략을 결정할 때 다양한 상품 카테고리에 집중하는 것이 단일 카테고리에 비해 잠재적으로 더 큰 이익을 가져다 줄 수 있음을 고려할 수 있습니다

In [6]:
# 11월 24일에, Top sellers 에 해당하는 애들이 무슨 카테고리 상품을 얼마씩 판거야?(순이익 계산)

# Convert Order_purchase_timestamp to datetime
orders['Order_purchase_timestamp'] = pd.to_datetime(orders['Order_purchase_timestamp'])

# Merge orders with order_items to calculate revenue per day
orders_items_merged = pd.merge(orders, order_items, on='Order_id')

# Calculate total revenue per day
daily_revenue = orders_items_merged.groupby(orders_items_merged['Order_purchase_timestamp'].dt.date)['Price'].sum().reset_index()

# Find the day with an unusually high revenue
daily_revenue['Difference'] = daily_revenue['Price'].diff().abs()
spike_day = daily_revenue[daily_revenue['Difference'] == daily_revenue['Difference'].max()]

# Filter data for the spike day: 2018-11-24
spike_day_data = orders_items_merged[orders_items_merged['Order_purchase_timestamp'].dt.date == pd.to_datetime("2018-11-24").date()]

# Aggregate data to find top sellers and products on this day
top_sellers = spike_day_data.groupby('Seller_id')['Price'].sum().sort_values(ascending=False).reset_index().head()

# Identify the top sellers from November 24, 2018
top_sellers_ids = top_sellers['Seller_id'].tolist()

# Filter the data for the top sellers from November 24, 2018
top_sellers_spike_day = spike_day_data[spike_day_data['Seller_id'].isin(top_sellers_ids)]

# Merge with product data to get product category information
top_sellers_categories = pd.merge(top_sellers_spike_day, products, on='Product_id')

# Calculate the revenue considering the quantity of each item sold
top_sellers_categories['Net_Revenue'] = top_sellers_categories['Price'] * top_sellers_categories['Order_item_id']

# Group data by seller and product category to sum up the net revenue
top_sellers_category_revenue = top_sellers_categories.groupby(['Seller_id', 'Product_category_name']).agg({
    'Net_Revenue': 'sum'
}).reset_index().sort_values(by='Net_Revenue', ascending=False)

top_sellers_category_revenue



,Seller_id,Product_category_name,Net_Revenue
3,SELLER_1667,agro_industry_and_commerce,20771.10
1,SELLER_0062,watches_gifts,5879.98
8,SELLER_2423,furniture_decor,5399.72
2,SELLER_0417,garden_tools,5390.00
4,SELLER_2194,bed_bath_table,4287.10
6,SELLER_2194,home_comfort,1150.80
7,SELLER_2423,construction_tools_lighting,750.00
0,SELLER_0062,electronics,199.99
5,SELLER_2194,furniture_decor,155.00


2018년 11월 24일, 블랙 프라이데이에 가장 많이 판매된 상위 셀러들이 각 카테고리별로 올린 순이익은 다음과 같습니다:

SELLER_1667
농업 산업 및 상업 (agro_industry_and_commerce): 순이익 20,771.10원
SELLER_0062
시계 및 선물 (watches_gifts): 순이익 5,879.98원
전자제품 (electronics): 순이익 199.99원
SELLER_2423
가구/장식 (furniture_decor): 순이익 5,399.72원
건설 도구/조명 (construction_tools_lighting): 순이익 750.00원
SELLER_0417
정원 도구 (garden_tools): 순이익 5,390.00원
SELLER_2194
침구/욕실용품 (bed_bath_table): 순이익 4,287.10원
홈 컴포트 (home_comfort): 순이익 1,150.80원
가구/장식 (furniture_decor): 순이익 155.00원
이 데이터는 특정 셀러들이 블랙 프라이데이에 특정 카테고리 상품에서 상당히 높은 매출을 달성했음을 보여줍니다. SELLER_1667은 특히 농업 산업 및 상업 카테고리에서 매우 높은 순이익을 올렸습니다. 이는 특정 이벤트나 할인 기간 동안 특정 상품군이 매출 증대에 크게 기여할 수 있음을 시사합니다.